In [9]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")

In [4]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [29]:
df = pd.read_parquet('/root/mlops_bootcamp/mlops-zoomcamp/homeworks/data/yellow_tripdata_2022-01.parquet')

### Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data

In [19]:
len(df.columns)

19

### Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [31]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
np.std(df.duration)

46.445295712725304

### Question 3: Fraction of the records left after dropping the outliers

In [8]:
df_without_outliers = df[(df.duration >= 1) & (df.duration <= 60)]

In [11]:
df_without_outliers.shape[0]/df.shape[0]

0.9827547930522406

In [32]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_27087/2069223988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


### Question 4: Dimensionality after OHE

In [33]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [34]:
len(dv.feature_names_)

516

## Model

In [37]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [38]:
df_train = read_dataframe('/root/mlops_bootcamp/mlops-zoomcamp/homeworks/data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('/root/mlops_bootcamp/mlops-zoomcamp/homeworks/data/yellow_tripdata_2022-02.parquet')

In [39]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [41]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Question 5: RMSE on train

In [44]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.001496179445599

### Question 6: RMSE on validation

In [45]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.795498122351478

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)